In [1]:
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv( "../../newSemana.csv" , encoding ="utf-8")
df["dates"] = df["dates"].apply(lambda x : x.replace("|" , "") )
df["article"] = df["article"].apply(lambda x : x.replace("\n" , "") )
df["article"] = df["article"].apply(lambda x : x.replace("\r" , "") )
df["article"] = df["article"].apply(lambda x : x.replace("\t" , "") )
#df["dates"] = df["dates"].apply(lambda x : x.replace(" " , "") )
df["dates"] = df["dates"].apply(lambda x : x.split(" ")[2] )
df.loc[ : , "dates"] = pd.to_datetime( df["dates"] , format = "%m/%d/%Y" , errors='coerce' )
df = df.dropna( subset = ["dates"])

In [3]:
df.head()

,Unnamed: 0,article,dates
0,0,Fueron ocho las personas condenadas este juev...,2015-04-09
2,2,"1.Oprima en la parte inferior derecha, el lug...",2014-01-30
3,3,Cuando Néstor Humberto Martínez intentó conven...,2016-06-15
4,4,SEMANA: ¿Qué es la novela negra?Rudolf Hommes:...,2014-11-22
5,5,"Desde julio de 2013, Claro, Movistar, Tigo y ...",2014-10-02


In [4]:
words_emb = {}

with open('../../SBW-vectors-300-min5.txt', 'r') as fopen:
    
    first_line = fopen.readline()
    next( fopen )
    
    for  n,line in enumerate(fopen):
        #data.append(line)
        
        t = line.split(" ")
        vec = t[1:]
        #print(vec)
        #print("wtff") 
        word = t[0].lower()
        vec = [ float( val ) for val in vec ]
        words_emb[word] = np.array(vec , np.float32 )
        if (n + 1) % 100000 == 0:
            print(word)
            print('done processed: ', n + 1)
            
print('done!')

escenográfico
done processed:  100000
concluded
done processed:  200000
nissl
done processed:  300000
octroyée
done processed:  400000
sheeqo
done processed:  500000
macroblastos
done processed:  600000
kamol
done processed:  700000
antonida
done processed:  800000
atent
done processed:  900000
vespucio
done processed:  1000000
done!


In [5]:
df18 = df[ df["dates"].dt.year == 2018 ]

In [6]:
df18.shape

(1576, 3)

In [7]:
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx 


stop_words = stopwords.words("spanish")

def getSentences( df_m ):
    # recibe el dataframe de un mes.
    sentences = []
    texto = ".".join( df_m["article"] ).lower()
    sentences = nltk.sent_tokenize( texto )
    sentences = [ x.replace("[^a-zA-Z]", " ")  for x in sentences ]
    sentences = [ x.replace(".", "")  for x in sentences ]
    
    clean_sent  = [ remove_stopwords(s) for s in sentences ]
    return sentences , clean_sent

def remove_stopwords(sen):
    tokens = sen.split(" ")
    
    sen_new = " ".join([i for i in tokens if i not in stop_words])          
    return sen_new

def getVectorSentence( clean_sentences ):
    sentence_vectors = []
    
    for i in clean_sentences:
        if len(i) != 0 :
            v = sum( [ words_emb.get( w , np.zeros((300,) )   ) for w in i.split(" ")    ])/len( i.split(" ") )
        else:
            v = np.zeros( (300,))
            
        sentence_vectors.append( v )
    return sentence_vectors 
 
def calcSimMatrix( vec_sentences ):
    
    M = np.zeros(  (  len(vec_sentences) , len(vec_sentences) ))
    for i in range(len(vec_sentences)): 
        for j in range(len(vec_sentences)): 
            if i != j: 
                  M[i][j] = cosine_similarity( vec_sentences[i].reshape(1,300),vec_sentences[j].reshape(1,300))[0,0]
                    
    return M

def textRank( M ):
    
    nx_graph = nx.from_numpy_matrix(M) 
    scores = nx.pagerank(nx_graph)
    return scores 
    
def getRank( scores , sentences , N = 10 ):
    
    ranked_sentences = sorted(  ((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    final = []
    for i in range(N): 
          final.append(ranked_sentences[i][1])
    return final

def process( df ):
    
    print("--getting sentences")
    sentences , clean_sent = getSentences( df )
    print("-- getting sentence vectors")
    vec_sentences = getVectorSentence( clean_sent )
    
    print("-- Calculating similarity matrix")
    M = calcSimMatrix( vec_sentences )
    
    print("-- Algoritm pageRank")
    scores = textRank(M)
    summ = getRank( scores, sentences , 10 )
    return summ
            

In [8]:
df18.shape

(1576, 3)

In [9]:
aa = process( df18[:20] )

--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank


In [10]:
months = set(  df18["dates"].dt.month )

In [11]:
months

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [12]:
results_final = []
for month in months:
    print( "Processing month: {}".format(month ) )
    dfm = df18[ df18["dates"].dt.month == month ]
    
    result = process( dfm.sample(20) )
    results_final.append( result )
    print("Done!")
 

Processing month: 1
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 2
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 3
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 4
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 5
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 6
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 7
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- Algoritm pageRank
Done!
Processing month: 8
--getting sentences
-- getting sentence vectors
-- Calculating similarity matrix
-- 

In [15]:
for m, r in enumerate( results_final ):
    print( "Mes:{}".format( m ) )
    for i in r[:3]:
        print(  "-"*10)
        print(i)
        

Mes:0
----------
por ahora, sánchez permanece en venezuela a la espera de su extradición al país, donde deberá responder por la violación de más de 500 niñospuede leer: los testigos contra el lobo feroz no quieren declarar los grupos armados organizados, gao, o más conocidos como disidencias, se convirtieron en el principal dolor de cabeza de las autoridadesesas bandas delincuenciales que no son otra cosa que los militantes de las farc que no aceptaron la firma del proceso de paz, y como tal ni se desmovilizaron y mucho menos entregaron sus armas, están detrás de los recientes hechos de violencia que enlutaron al suroeste del paísla mayoría de esas disidencias se concentran en departamentos como nariño, cauca y valle, donde se calcula se desmovilizaron 3000 subversivos que integraban los míticos frentes 6, 29 y 30 que formaban el temido bloque occidental de las farcpuede leer: disidencia de las farc se van o se quedande ahí que desde el año pasado los colombianos vienen escuchando de n

In [64]:
for i in results:
    print("-----------------")
    print( i )

-----------------
también dejó 44 personas heridas, 12089 evacuadas de las cuales 3319 fueron llevadas a albergues, según precisa un balance dado este miércoles por la coordinadora para la reducción de desastres (conred), ente a cargo de la protección civila pesar de la condiciones adversas, cientos de rescatistas, policías y militares ingresaron a la ahora llamada "zona 0" para reanudar las tareas de búsqueda de desaparecidosun grupo de agentes de policía fue designado para marcar con tinta roja las viviendas que ya fueron revisadas en medio de la búsqueda de desaparecidosun nuevo deslave del coloso, de 3763 metros de altura y situado 35 km al suroeste de la capital, obligó la tarde del martes a suspender las labores de rescate de víctimas y siete comunidades de escuintla fueron evacuadas
-----------------
casi 200 desaparecidos"ya tenemos un dato con nombres y de las localidades donde hay personas desaparecidas y tenemos el número que son 192", dijo en rueda de prensa el secretario d

In [33]:
sentences , clean_sent = getSentences( df18[:10] ) 

In [34]:
sentences[0]

'la participación comunitaria es clave en el desarrollo de una ciudad'

In [35]:
clean_sent[0]

'participación comunitaria clave desarrollo ciudad'

In [36]:
len( sentences )

172

In [38]:
aa = getVectorSentence( clean_sent )

In [39]:
len(aa)

172

In [41]:
aa[0]

array([ 1.09147802e-01, -5.10328040e-02,  3.25959967e-03, -4.04644012e-02,
        4.18063961e-02, -9.35776010e-02, -2.37913989e-02, -5.26791997e-02,
        2.97995992e-02,  5.90199977e-03,  1.45914005e-02,  1.94760002e-02,
       -2.59681996e-02,  8.47559981e-03, -9.20999981e-03,  1.38991997e-02,
       -1.96471997e-02,  3.57517973e-02,  8.64100084e-03,  1.47193996e-02,
       -4.04519998e-02, -7.56088048e-02, -2.80644037e-02,  2.92622000e-02,
       -5.01572005e-02,  1.95996016e-02, -1.02007605e-01,  5.67450002e-02,
       -4.85080015e-03,  2.00213995e-02,  2.90832017e-02, -2.72318013e-02,
       -1.22296000e-02,  2.03514006e-02,  2.08784007e-02,  4.26991992e-02,
       -4.10230011e-02, -5.00921980e-02, -6.84002042e-02, -2.41908003e-02,
        1.67753994e-02,  8.37909952e-02, -4.30670008e-02, -1.00653991e-02,
       -3.48439999e-02, -3.00463997e-02,  1.44697996e-02, -2.78160013e-02,
       -3.93658020e-02,  3.45095992e-02, -2.84419977e-03,  7.73780048e-03,
       -3.29587981e-02,  